In [1]:
import pandas as pd
from datetime import datetime
import time
import json
from math import radians, cos, sin, asin, sqrt

In [2]:
def lldis(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

In [3]:
raw = pd.read_csv('data/train.tsv', sep='\t')

In [4]:
raw[:5]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [5]:
raw['category_id'].value_counts()

0     2544488
1     1293227
2     1281052
3     1233523
4      931638
5      787817
6      772039
7      720987
8      569545
10     543662
9      538951
11     445998
12     434892
13     429443
14     366025
15     320721
16     303283
17     257026
19     239080
18     215859
20     125426
22     102039
21      97935
23      94685
24      69860
25      65579
26      64628
27      39559
28      38358
29      32863
30      28701
31       9216
32       1895
Name: category_id, dtype: int64

In [6]:
raw['quantity'].value_counts()

1      14216142
2        522973
3        116993
4         61183
5         29698
         ...   
305           1
109           1
148           1
79            1
175           1
Name: quantity, Length: 147, dtype: int64

In [7]:
parsed = raw[['record_number', 'shipment_method_id', 'shipping_fee', 'carrier_min_estimate', 'carrier_max_estimate', 'category_id', 'item_price', 'quantity']]

In [8]:
parsed.shape

(15000000, 8)

In [9]:
dict1 = {
    'B2C': 0,
    'C2C': 1
}

parsed['bt'] = raw['b2c_c2c'].map(dict1)
parsed[:10]

/tmp/ipykernel_5123/1629042217.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed['bt'] = raw['b2c_c2c'].map(dict1)


,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt
0,1,0,0.00,3,5,13,27.95,1,0
1,2,0,3.00,3,5,0,20.50,1,1
2,3,0,4.50,3,5,1,19.90,1,0
3,4,0,0.00,3,5,1,35.50,1,0
4,5,0,0.00,3,5,1,25.00,1,0
5,6,0,0.00,3,5,3,10.39,1,0
6,7,0,0.00,3,5,11,5.70,1,0
7,8,0,2.95,3,5,1,6.00,1,0
8,9,3,0.00,2,8,18,5.55,1,0
9,10,1,0.00,2,5,13,59.98,1,0


In [10]:
parsed.shape

(15000000, 9)

In [11]:
'2019-03-26 15:11:00.000-07:00'.split()[1][:2]

'15'

In [12]:
int('2019-03-26 15:11:00.000-07:00'.split()[1][-6:-3])

-7

In [13]:
cdate = datetime.strptime('2019-03-26', "%Y-%m-%d")
cdate.month

3

In [14]:
def add_func(row):
    acct = row['acceptance_scan_timestamp']
    dd = row['delivery_date']
    arr = acct.split()
    cdate = datetime.strptime(arr[0], "%Y-%m-%d")
    ddate = datetime.strptime(dd, "%Y-%m-%d")
    delta = ddate - cdate
    
    acc_hour = arr[1][:2]
    tz = int(arr[1][-6:-3])
    if cdate.month >= 3 and cdate.month <= 11:
        isdst = 0
    else:
        isdst = 1
        
    return delta.days, acc_hour, tz, isdst

In [15]:
fattr = raw.apply(add_func, axis=1, result_type='expand')

In [16]:
fattr

,0,1,2,3
0,3,15,-7,0
1,3,12,-7,0
2,3,16,-5,1
3,4,16,-8,1
4,3,16,-7,0
...,...,...,...,...
14999995,3,15,-5,0
14999996,6,14,-4,0
14999997,5,15,-7,0
14999998,3,10,-5,1


In [17]:
parsed['target'] = fattr[0]
parsed['acc_hour'] = fattr[1]
parsed['sender_tz'] = fattr[2]
parsed['isdst'] = fattr[3]

/tmp/ipykernel_5123/946312585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed['target'] = fattr[0]
/tmp/ipykernel_5123/946312585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed['acc_hour'] = fattr[1]
/tmp/ipykernel_5123/946312585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [18]:
raw['sender_tz'] = fattr[2]
raw['isdst'] = fattr[3]

In [19]:
parsed[:5]

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt,target,acc_hour,sender_tz,isdst
0,1,0,0.0,3,5,13,27.95,1,0,3,15,-7,0
1,2,0,3.0,3,5,0,20.50,1,1,3,12,-7,0
2,3,0,4.5,3,5,1,19.90,1,0,3,16,-5,1
3,4,0,0.0,3,5,1,35.50,1,0,4,16,-8,1
4,5,0,0.0,3,5,1,25.00,1,0,3,16,-7,0


In [20]:
parsed[parsed['target'] < 0]

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt,target,acc_hour,sender_tz,isdst
1558,1559,0,0.00,3,5,13,3.76,1,0,-2,18,-6,1
1777,1778,3,3.79,2,8,3,6.95,1,0,-6,17,-4,0
3928,3929,0,0.00,3,5,3,10.00,1,0,-43,20,-7,0
7233,7234,0,0.00,3,5,5,14.00,1,0,-1,21,-4,0
7642,7643,0,3.50,3,5,2,32.00,1,1,-10,09,-7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994475,14994476,0,0.00,3,5,2,22.99,1,0,-2,19,-4,0
14996746,14996747,0,0.00,3,5,1,14.88,1,0,-21,17,-7,0
14997239,14997240,2,9.26,2,9,0,18.00,1,0,-3,20,-5,1
14998005,14998006,1,0.00,2,5,1,340.00,1,0,-16,17,-5,0


In [21]:
zip_info = json.load(open('data/zipcode_dict.json', 'r'))

In [22]:
def add_func(row):
    sender_tz = row['sender_tz']
    sender_zip = str(row['item_zip'])
    re_zip = str(row['buyer_zip'])
    
    sender_info = zip_info[sender_zip]
    re_info = zip_info[re_zip]
    
    if re_info['tz'] != False:
        tz_dis = re_info['tz'] - sender_tz - int(row['isdst'])
    else:
        tz_dis = 0
    
    if sender_info['lat'] != False and re_info['lat'] != False:
        dis = int(lldis(sender_info['lat'], re_info['lat'], sender_info['lng'], re_info['lng']) / 1000)
    else:
        dis = -1
    
    if sender_info['city'] != False and re_info['city'] != False:
        if sender_info['city'] == re_info['city']:
            cross_city = 0
        else:
            cross_city = 1
    else:
        cross_city = -1
    
    if sender_info['state'] != False and re_info['state'] != False:
        if sender_info['state'] == re_info['state']:
            cross_state = 0
        else:
            cross_state = 1
    else:
        cross_state = -1
    
    return tz_dis, dis, cross_city, cross_state

In [23]:
dis_attr = raw.apply(add_func, axis=1, result_type='expand')

In [24]:
dis_attr

,0,1,2,3
0,2,3,1,1
1,1,1,1,1
2,-2,1,1,1
3,0,1,1,1
4,1,2,1,1
...,...,...,...,...
14999995,0,1,1,1
14999996,-1,0,1,1
14999997,1,2,1,1
14999998,0,-1,-1,-1


In [30]:
parsed['tz_dis'] = dis_attr[0]
parsed['dis'] = dis_attr[1]
parsed['cross_city'] = dis_attr[2]
parsed['cross_state'] = dis_attr[3]

/tmp/ipykernel_5123/3186594668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed['tz_dis'] = dis_attr[0]
/tmp/ipykernel_5123/3186594668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsed['dis'] = dis_attr[1]
/tmp/ipykernel_5123/3186594668.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [31]:
dis_attr[0].value_counts()

-1     5199652
 0     2253792
-2     2128312
 2     1813210
 1     1371500
-4     1111304
-3      998387
 3      122879
 5         350
 4         332
-5          80
 13         57
-6          40
 14         36
 16         30
 15         20
 12          9
 17          7
-7           2
 9           1
Name: 0, dtype: int64

In [32]:
parsed

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt,target,acc_hour,sender_tz,isdst,tz_dis,dis,cross_city,cross_state
0,1,0,0.00,3,5,13,27.95,1,0,3,15,-7,0,2,3,1,1
1,2,0,3.00,3,5,0,20.50,1,1,3,12,-7,0,1,1,1,1
2,3,0,4.50,3,5,1,19.90,1,0,3,16,-5,1,-2,1,1,1
3,4,0,0.00,3,5,1,35.50,1,0,4,16,-8,1,0,1,1,1
4,5,0,0.00,3,5,1,25.00,1,0,3,16,-7,0,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,14999996,1,9.85,2,5,3,7.35,1,0,3,15,-5,0,0,1,1,1
14999996,14999997,2,0.00,2,9,1,36.00,1,1,6,14,-4,0,-1,0,1,1
14999997,14999998,2,2.99,2,9,0,7.99,1,0,5,15,-7,0,1,2,1,1
14999998,14999999,0,0.00,3,5,1,34.99,1,0,3,10,-5,1,0,-1,-1,-1


In [ ]:
parsed = parsed[parsed['target'] >= 0]